In [12]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import sys
sys.path.append('../')
import yaml
import logging
import requests
from pprint import pprint
import json
import time
import os
from pymongo import MongoClient
import dateparser

In [10]:
mongourl = 'mongodb://localhost:27017/'
MONGO_CLIENT = MongoClient(mongourl)

In [14]:
collection = MONGO_CLIENT["news"]["article_en"]
not_processed_docs = collection.find()

<IPython.core.display.Javascript object>

In [ ]:
for document in not_processed_docs:
    